## Translate Wyoming stream gage data into Geoconnex


Station data came from Brent via email:

Here is a list of the locations in seoflow.wyo.gov with the name, ID, Lat and Long.  Both surface water and groundwater locations are included.
It is an Excel 2022 spreadsheet.  Can't do CSV as some of the names have commas. 
Brent
brent.wickham@wyo.gov
**Note: Shell Creek above Lake DeSmet in the spreadsheet incorrectly had a positive latitude, and there's no data for it, so it was removed in a block at the end.

Other data comes from: https://seoflow.wyo.gov/Data/Map/Parameter/Discharge/Statistic/LATEST/Interval/Latest and was exported as a csv. 


In [28]:
import arcpy

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\ewiggans\Desktop\GeoconnexNPDES\GeoconnexNPDES\GeoconnexNPDES.gdb"

# Define the input layer
WY_gage = "WyomingStreamGages"

# Define a list of field names and their types
field_list = [
    ("uri", "TEXT"),
    ("location_url", "TEXT"),
    ("name", "TEXT"),
    ("id", "TEXT"),
    ("provider_name", "TEXT"),
    ("provider_id", "TEXT"),
    ("provider_url", "TEXT"),
    ("provider_code", "TEXT"),
    ("mainstem_uri", "TEXT"),
    ("comid", "TEXT")
]

# Add the new fields using a loop
for field_name, field_type in field_list:
    arcpy.AddField_management(WY_gage, field_name, field_type)

# Create an update cursor to delete rows with NULL 'latitude' or 'longitude'
with arcpy.da.UpdateCursor(WY_gage, ['latitude', 'longitude']) as cursor:
    for row in cursor:
        latitude, longitude = row
        if latitude is None or longitude is None:
            cursor.deleteRow()

# Confirm the field addition
field_names = [field.name for field in arcpy.ListFields(WY_gage)]
print("Fields in the layer after addition:", field_names)





Fields in the layer after addition: ['OBJECTID', 'Shape', 'LocationName', 'Identifier', 'Latitude', 'Longitude', 'uri', 'location_url', 'name', 'id', 'provider_name', 'provider_id', 'provider_url', 'provider_code', 'mainstem_uri', 'comid']


In [29]:
# Confirm unique ID in identifier field
# Field to check for uniqueness
field_name = "Identifier"

# Use a set to keep track of unique values encountered in the field
unique_values = set()

# Initialize a counter to keep track of the total number of rows
total_rows = 0

# Initialize a flag to check for spaces
has_spaces = False

# Start a search cursor to iterate through the rows and check for unique values and spaces
with arcpy.da.UpdateCursor(WY_gage, field_name) as cursor:
    for row in cursor:
        total_rows += 1
        value = row[0]
        if value in unique_values:
            print(f"Non-unique value found: {value}")
        else:
            unique_values.add(value)

        if " " in value:
            has_spaces = True
            # Replace spaces with "-"
            row[0] = value.replace(" ", "-")
            cursor.updateRow(row)

# Check if the number of unique values is the same as the total number of rows
if len(unique_values) == total_rows:
    print("All values in the 'Identifier' field are unique.")
else:
    print("There are duplicate values in the 'Identifier' field.")

# Check if there are any spaces in the 'Identifier' field
if has_spaces:
    print("There were spaces in the 'Identifier' field. They have been replaced with '-'.")
else:
    print("No spaces found in the 'Identifier' field.")


All values in the 'Identifier' field are unique.
There were spaces in the 'Identifier' field. They have been replaced with '-'.


In [46]:

#set Identifier field to id and name
#fields to update
id_field = "id"
name_field = "name"
field_name = "Identifier"

# Calculate the 'id' field and 'name' field using the 'Identifier' field
expression = "!" + field_name + "!"
codeblock = ""

# Use the CalculateField tool to perform the field calculation
arcpy.CalculateField_management(WY_gage, id_field, expression, "PYTHON3", codeblock)
arcpy.CalculateField_management(WY_gage, name_field, expression, "PYTHON3", codeblock)

#print("Updated 'id' and 'name' fields with values from the 'Identifier' field.")

arcpy.CalculateField_management(WY_gage, "provider_name", "'Wyoming State Engineers Office'")
arcpy.CalculateField_management(WY_gage, "provider_url", "'https://seo.wyo.gov/'")
arcpy.CalculateField_management(WY_gage, "provider_id", "!" + field_name +"!")
arcpy.CalculateField_management(WY_gage, "provider_code", "'wyseo'")


<Result 'WyomingStreamGages'>

In [47]:
#Calculate URI 
codeblock = """
def url_join(*parts: list) -> str:
    return '/'.join([str(p).strip().strip('/') for p in parts])
"""
## pattern for uri is: geoconnex.us/iow/wyseo/gages/id

expression = """url_join("https://geoconnex.us/wyseo/gages", !id!)"""
arcpy.management.CalculateField(WY_gage, "uri", expression, "PYTHON3", codeblock)
print("uri calculated")



uri calculated


In [48]:
##Read in the download table

In [49]:
import pandas as pd
pd.set_option('display.width', 1000)

# Path to the CSV file
csv_file_path = r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\List-20230801130904_WYgages.csv"


# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the data frame to verify the data
print(df.head())


                               Data_Set_Id  ... Status
0  Discharge.Historical.Discharge@010266RS  ...      -
1             Discharge.Discharge@010266RS  ...      -
2         Discharge.Discharge.MDT@010266RS  ...      -
3     Discharge.Discharge.MDT.MDQ@010266RS  ...      -
4         Discharge.Discharge.MDT@0102BOUT  ...      -

[5 rows x 7 columns]


In [50]:
#View column headers 
df.columns

Index(['Data_Set_Id', 'Location', 'LocationFolder', 'StartofRecord', 'EndofRecord', 'Value', 'Status'], dtype='object')

In [51]:
# Perform the field calculations on the pandas DataFrame
df["location_id"] = df["Data_Set_Id"].str.replace(r".*@", "", regex=True)
df["about_url"] = "https://geoconnex.us/wyseo/gages/" + df["location_id"]
df["data_set_type"] = df["Data_Set_Id"].str.replace(r"@.*", "", regex=True)
df["data_set_path"] = df["data_set_type"].str.replace(".", "/", 1)
df["url"] = "https://seoflow.wyo.gov/Data/DataSet/Summary/Location/" + df["location_id"] + "/DataSet/" + df["data_set_path"]
df["provider_code"] = "wyseo"
df["parameter_name"] = df["data_set_type"]
df["parameter_group"] = "discharge"
df["parameter_id"] = df["provider_code"] + "-" + df["parameter_group"]
df["about_uri"] = df["about_url"]
df["name"] = df["Data_Set_Id"]



In [52]:
df.head()

,Data_Set_Id,Location,LocationFolder,StartofRecord,EndofRecord,Value,Status,location_id,about_url,data_set_type,data_set_path,url,provider_code,parameter_name,parameter_group,parameter_id,about_uri,name
0,Discharge.Historical.Discharge@010266RS,66 Reservoir Supply Ditch,All Locations.Division 1.District 02,10/1/2017 6:00,10/21/2020 18:00,NaN,-,010266RS,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Historical.Discharge,Discharge/Historical.Discharge,https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Historical.Discharge,wyseo,Discharge.Historical.Discharge,discharge,wyseo-discharge,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Historical.Discharge@010266RS
1,Discharge.Discharge@010266RS,66 Reservoir Supply Ditch,All Locations.Division 1.District 02,10/1/2017 6:00,6/2/2022 2:00,NaN,-,010266RS,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge,Discharge/Discharge,https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge,wyseo,Discharge.Discharge,discharge,wyseo-discharge,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge@010266RS
2,Discharge.Discharge.MDT@010266RS,66 Reservoir Supply Ditch,All Locations.Division 1.District 02,10/1/2017 6:00,6/2/2022 2:00,NaN,-,010266RS,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge.MDT,Discharge/Discharge.MDT,https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge.MDT,wyseo,Discharge.Discharge.MDT,discharge,wyseo-discharge,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge.MDT@010266RS
3,Discharge.Discharge.MDT.MDQ@010266RS,66 Reservoir Supply Ditch,All Locations.Division 1.District 02,10/1/2017 6:00,6/1/2022 6:00,NaN,-,010266RS,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge.MDT.MDQ,Discharge/Discharge.MDT.MDQ,https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge.MDT.MDQ,wyseo,Discharge.Discharge.MDT.MDQ,discharge,wyseo-discharge,https://geoconnex.us/wyseo/gages/010266RS,Discharge.Discharge.MDT.MDQ@010266RS
4,Discharge.Discharge.MDT@0102BOUT,Goshen Reservoir Outlet,All Locations.Division 1.District 02,10/1/2017 6:00,8/28/2022 16:30,NaN,-,0102BOUT,https://geoconnex.us/wyseo/gages/0102BOUT,Discharge.Discharge.MDT,Discharge/Discharge.MDT,https://seoflow.wyo.gov/Data/DataSet/Summary/Location/0102BOUT/DataSet/Discharge/Discharge.MDT,wyseo,Discharge.Discharge.MDT,discharge,wyseo-discharge,https://geoconnex.us/wyseo/gages/0102BOUT,Discharge.Discharge.MDT@0102BOUT


In [53]:
# Print the first five rows of the "data_set_url" column
pd.options.display.max_colwidth = 300
print(df["url"].head())

0    https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Historical.Discharge
1               https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge
2           https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge.MDT
3       https://seoflow.wyo.gov/Data/DataSet/Summary/Location/010266RS/DataSet/Discharge/Discharge.MDT.MDQ
4           https://seoflow.wyo.gov/Data/DataSet/Summary/Location/0102BOUT/DataSet/Discharge/Discharge.MDT
Name: url, dtype: object


In [54]:
#Perform final field calculations on pandas DataFrame

end_df = df[["about_uri", "url", "name", "provider_code", "parameter_id", "parameter_name", "parameter_group"]]
end_df.head()

#Export and write to new CSV

end_df.to_csv("C:\\Users\\ewiggans\\Desktop\\GeoConnexMap\\WyomingGage\\WY_download_table.csv")

In [55]:
import json
import requests
from shapely.geometry import shape, Point

def get_comid_intersect(geom):
    # Convert the input geom to GeoJSON using Shapely
    point = Point(geom)

    # Convert the Point to GeoJSON
    geom_geojson = shape(point).__geo_interface__

    url = 'https://nhdpv2-census.internetofwater.app/collections/2020/items?filter-lang=cql-json'
    filter_ = {
        'intersects': [
            {'property': 'shape'},
            geom_geojson  # Use the Shapely-converted GeoJSON
        ]
    }
    headers = {
        'Content-Type': 'application/query-cql-json'
    }
    r = requests.post(url, headers=headers, json=filter_)
    fc = r.json()
    if 'features' in fc and len(fc['features']) > 0:
        feature = fc['features'][0]
        return feature['properties']['featureid']
    else:
        return None

# Update Cursor
with arcpy.da.UpdateCursor(WY_gage, ["Shape", "comid"]) as cursor:
    for row in cursor:
        geom = row[0]
        comid = get_comid_intersect(geom)
        if comid is not None:
            print(comid, end='\r', flush=True)
            row[1] = comid
            cursor.updateRow(row)

# Complete
print("comid calculated")

comid calculated


In [56]:
# Read in the CSV file as a geodatabase table
csv_table = r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\nhdpv2_lookup.csv"
csv_table_name = "NHDPV2_Lookup"
arcpy.TableToTable_conversion(csv_table, arcpy.env.workspace, csv_table_name)

# Add a new field 'comid_text' to the CSV table with a data type of TEXT
arcpy.AddField_management(csv_table_name, "comid_text", "TEXT")

# Calculate the 'comid_text' field by copying the values from the 'comid' field
expression = "!comid!"
codeblock = ""
arcpy.CalculateField_management(csv_table_name, "comid_text", expression, "PYTHON3", codeblock)

print("Added 'comid_text' field and set its values equal to 'comid' as text.")


Added 'comid_text' field and set its values equal to 'comid' as text.


In [57]:
# Join the CSV table to the "WyomingStreamGage" feature class based on the "comid" field
arcpy.AddJoin_management(WY_gage, "comid", csv_table_name, "comid_text", "KEEP_COMMON")

# Calculate the "mainstem_uri" field to be equal to the "uri" field in the CSV table
expression = "!{}.uri!".format(csv_table_name)
arcpy.CalculateField_management(WY_gage, "mainstem_uri", expression, "PYTHON3")

# Remove the join to the CSV table
arcpy.RemoveJoin_management(WY_gage, csv_table_name)
print("complete")


complete


In [58]:
## Fix lat/long showing up in China
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=WY_gage,
    selection_type="NEW_SELECTION",
    where_clause="Identifier = '0209SCLD'",
    invert_where_clause=None
)
print("Record with incorrect lat/long selected")

arcpy.management.DeleteRows(
    in_rows="WyomingStreamGages"
)
print("Bad record deleted")

arcpy.SelectLayerByAttribute_management(WY_gage, "CLEAR_SELECTION")


Record with incorrect lat/long selected
Bad record deleted


<Result 'WyomingStreamGages'>

In [59]:
# Define the output GeoJSON file path
output_geojson_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\WY_gage_export.geojson"

# Export the feature class to GeoJSON
arcpy.FeaturesToJSON_conversion(in_features = WY_gage, 
                                out_json_file = output_geojson_file,
                                geoJSON = "GEOJSON")


print(f"Exported {WY_gage} to {output_geojson_file} in GeoJSON format.")


Exported WyomingStreamGages to C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\WY_gage_export.geojson in GeoJSON format.


In [67]:
import geopandas as gpd

def group_data(gdf, gpkg_file, csv_file):
    # Make mapping dictionary
    mapping = {}
    df = pd.read_csv(csv_file)
    for index, row in df.iterrows():
        location = row['about_uri']
        data = row.drop(['about_uri'])
        if location not in mapping:
            mapping[location] = []
        mapping[location].append(data.to_dict())
    # Apply mapping
    for index, row in gdf.iterrows():
        location = row['uri']
        if location in mapping:
            gdf.at[index, 'data'] = \
                json.dumps(mapping[location])
    gdf.to_file(gpkg_file, driver="GPKG")

gdf = gpd.read_file(r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\WY_gage_export.geojson")
csv_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\WY_download_table.csv"
gpkg_file = r"C:\Users\ewiggans\Desktop\GeoConnexMap\WyomingGage\wy_gages.gpkg"
group_data(gdf, gpkg_file, csv_file)
print("Complete")

Complete
